In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

Data Preprocessing

training set

In [3]:
from numpy.ma import indices

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_set = train_datagen.flow_from_directory("train",
                                              target_size=(64, 64),
                                              batch_size=32,
                                              class_mode="categorical")


Found 557 images belonging to 2 classes.


test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory("test",
                                                        target_size=(64, 64),
                                                        batch_size=32,
                                                        class_mode="categorical")

Found 140 images belonging to 2 classes.


Building convolutional neural network

In [5]:
cnn = tf.keras.models.Sequential()

1st Convolutional layer

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=[64, 64, 3]))

Pooling layer, applying max pooling

In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

2nd Convolutional and pooling layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Flattening layer

In [9]:
cnn.add(tf.keras.layers.Flatten())

Full connection layers

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))

Output layer

In [11]:
cnn.add(tf.keras.layers.Dense(units=2, activation="softmax"))

Compiling

In [12]:
cnn.compile(optimizer="adam",loss="categorical_crossentropy",
                  metrics=["accuracy"])

Training the model and evaluating

In [13]:
cnn.fit(x=train_set, validation_data=test_set ,epochs=35)

Epoch 1/35
18/18 [==============================] - 10s 464ms/step - loss: 0.8069 - accuracy: 0.5368 - val_loss: 0.6948 - val_accuracy: 0.4286
Epoch 2/35
18/18 [==============================] - 7s 405ms/step - loss: 0.6943 - accuracy: 0.5135 - val_loss: 0.6923 - val_accuracy: 0.4857
Epoch 3/35
18/18 [==============================] - 8s 425ms/step - loss: 0.6880 - accuracy: 0.5853 - val_loss: 0.6948 - val_accuracy: 0.5143
Epoch 4/35
18/18 [==============================] - 7s 403ms/step - loss: 0.6791 - accuracy: 0.5799 - val_loss: 0.6973 - val_accuracy: 0.5000
Epoch 5/35
18/18 [==============================] - 7s 397ms/step - loss: 0.6710 - accuracy: 0.5996 - val_loss: 0.6901 - val_accuracy: 0.5429
Epoch 6/35
18/18 [==============================] - 7s 396ms/step - loss: 0.6952 - accuracy: 0.5871 - val_loss: 0.6885 - val_accuracy: 0.5571
Epoch 7/35
18/18 [==============================] - 7s 399ms/step - loss: 0.6691 - accuracy: 0.6068 - val_loss: 0.6789 - val_accuracy: 0.5571
Epoch

Making single prediction

In [21]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

# Load the trained model


# Load the image and resize it
img = cv2.imread("Thinking-of-getting-a-cat.png")
orig_img = cv2.resize(img, (500, 500))

# Preprocess the image
img = cv2.resize(img, (64, 64))
img = tf.keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0

# Predict the class of the image
prediction = cnn.predict(img)
class_names = ['cat', 'dog']
class_idx = np.argmax(prediction)

#display the class and probability
if class_idx == 0:
    color = (0, 255, 0)  # green
else:
    color = (0, 0, 255)  # red
probability = prediction[0][class_idx] * 100
text = f"{class_names[class_idx]}: {probability:.2f}%"
cv2.putText(orig_img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
cv2.imshow("Prediction", orig_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 196ms/step
